In [1]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

# example mina explorer queries

In [22]:
from MinaClient import Client
from mina_schemas import mina_explorer_schema
from sgqlc.operation import Operation
import pandas as pd

In [23]:
MINA_EXPLORER_ENDPOINT = "https://graphql.minaexplorer.com/"
mina_explorer_client = Client(endpoint=MINA_EXPLORER_ENDPOINT)

## stakes

In [27]:
STAKER_PUBLIC_KEY = "B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h1o7pvLPAN" # mina explorer
LEDGER_HASH = "jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4thw2Ee"

In [28]:
op = Operation(mina_explorer_schema.Query)

query = mina_explorer_schema.StakeQueryInput(
    delegate=STAKER_PUBLIC_KEY, ledger_hash=LEDGER_HASH
)

stakes = op.stakes(query=query, limit=1000)

stakes.public_key()
stakes.balance()

res = mina_explorer_client.send_any_query(op)

stakes_df = pd.json_normalize(res["data"]["stakes"], sep="_")

In [29]:
stakes_df.head()

,balance,public_key
0,6697.0,B62qkbdgRRJJfqcyVd23s9tgCkNYuGMCmZHKijnJGqYgs9...
1,32558.0,B62qqMo7X8i2NnMxrtKf3PAWfbEADk4V1ojWhGeH6Gvye9...
2,7000.0,B62qkrhfb1e3fV2HCsFxvnjKp1Yu4UyVpytucWDW16Ri3r...
3,19200.0,B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F8...
4,65328.0,B62qqrn3yzWRDJrUni6cRva4t51AcnY1o1pM4xpB78MfHU...


In [40]:
epoch = 0
block_height_gte = 0
block_height_lte = 5075

consensus_state = mina_explorer_schema.BlockProtocolStateConsensusStateQueryInput(
    epoch=epoch
)
protocol_state = mina_explorer_schema.BlockProtocolStateQueryInput(
    consensus_state=consensus_state
)

op = Operation(mina_explorer_schema.Query)

query = mina_explorer_schema.BlockQueryInput(
    block_height_gte=block_height_gte,
    block_height_lte=block_height_lte,
    creator=STAKER_PUBLIC_KEY,
    protocol_state=protocol_state,
)

blocks = op.blocks(
    query=query,
    limit=500,
)

blocks.block_height()
blocks.canonical()
blocks.tx_fees()
blocks.snark_fees()
blocks.date_time()
    
res = mina_explorer_client.send_any_query(op)

In [41]:
pd.DataFrame(res['data']['blocks'])

,blockHeight,canonical,dateTime,snarkFees,txFees
0,5044,True,2021-03-31T19:00:00Z,0,30000000
1,5002,False,2021-03-31T16:15:00Z,0,20000000
2,4960,False,2021-03-31T13:03:00Z,0,21000000
3,4933,True,2021-03-31T11:15:00Z,0,20000000
4,4863,True,2021-03-31T06:18:00Z,0,31000000
...,...,...,...,...,...
145,160,False,2021-03-17T11:24:00Z,0,10000000
146,139,True,2021-03-17T09:48:00Z,0,20000000
147,68,False,2021-03-17T04:33:00Z,0,10000000
148,39,True,2021-03-17T02:24:00Z,0,10000000
